In [368]:
import pdal 
from osgeo import gdal, osr
import os 
import numpy as np

gdal.UseExceptions()

In [369]:
basedir = "/media/riannek/minimax/gleis"
run24 = os.path.join(basedir, "2024-08-13/01/run24/01")
run14 = os.path.join(basedir, "2024-08-14/01/run14/01")

In [393]:
filename = "4473900_5335875.copc.laz"
#filename = "4473925_5335900.copc.laz"
filename=  "4473550_5338225.copc.laz"


run = run14

voxel_size = 1

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

In [394]:
pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

In [395]:
# Get EPSG code

def get_reader_name(metadata):
    for key in metadata.keys():
        if key.startswith("readers"):
            return key
    return None

metadata = pipeline.metadata['metadata']
reader_name = get_reader_name(metadata)
epsg = metadata[reader_name]['srs']['json']['id']['code']
epsg 

5684

In [396]:
xy = np.vstack((points['X'], points['Y'])).T

In [399]:
xy 

array([[4473574.93146878, 5338248.09948195]])

In [397]:
maxp = xy.max(axis=0)
minp = xy.min(axis=0)
maxp, minp

(array([4473574.93146878, 5338248.09948195]),
 array([4473574.93146878, 5338248.09948195]))

In [398]:
num_vox = np.ceil((maxp - minp) / voxel_size).astype(int) # Anzahl der Voxel in jeder Dimension
num_vox  

array([0, 0])

In [376]:
voxel_area = voxel_size * voxel_size

In [400]:
voxels = ((xy - minp) // voxel_size).astype(int)

In [378]:
voxels[:,0] = np.clip(voxels[:,0], 0, num_vox[0]-1)
voxels[:,1] = np.clip(voxels[:,1], 0, num_vox[1]-1)

In [402]:
voxel_key, point_count = np.unique(voxels, axis=0, return_counts=True)

In [380]:
density = point_count / voxel_area

In [404]:
max_x = voxel_key[:, 0].max()
max_y = voxel_key[:, 1].max()

max_x, max_y

(np.int64(0), np.int64(0))

In [382]:
voxel_key[:, 0].max(), voxel_key[:, 1].max()

(np.int64(24), np.int64(24))

In [383]:
raster = np.zeros((max_x + 1, max_y + 1), dtype=int)


np.add.at(raster, (voxel_key[:, 0], voxel_key[:, 1]), point_count)

In [384]:
# Scheint nötig zu sein bei Gauss-Krüger
raster = np.flip(raster.T, axis=0)

In [385]:
np.ceil((maxp - minp) / voxel_size).astype(int)

array([25, 25])

In [386]:
voxel_key[:, 0].max(), voxel_key[:, 1].max() 

(np.int64(24), np.int64(24))

In [387]:
# geotransform = (minp[0], voxel_size, 0, maxp[1], 0, voxel_size)
geotransform = (minp[0], voxel_size, 0, maxp[1], 0, -voxel_size)


In [388]:
geotransform 

(np.float64(4473899.999975428), 1, 0, np.float64(5335899.999986927), 0, -1)

In [389]:
minp[0]

np.float64(4473899.999975428)

In [390]:
# Erstelle das GeoTIFF
output_filename = "density.tif"

driver = gdal.GetDriverByName("GTiff")
rows, cols = raster.shape
dataset = driver.Create(output_filename, cols, rows, 1, gdal.GDT_Int32)

dataset.SetGeoTransform(geotransform)

srs = osr.SpatialReference()
srs.ImportFromEPSG(epsg)
dataset.SetProjection(srs.ExportToWkt())

band = dataset.GetRasterBand(1)
band.WriteArray(raster)
band.SetNoDataValue(0)

# Schließe das Dataset
# dataset.FlushCache()
dataset = None